In [1]:
using Mosek, JuMP, OffsetArrays, LinearAlgebra , MosekTools, Ipopt, Printf

In [2]:
## elementary operations

function e_i(n, i)
    e_i_vec = zeros(n, 1)
    e_i_vec[i] = 1
    return e_i_vec
end

function ⊙(a,b)
    return ((a*b') .+ transpose(a*b')) ./ 2
end

## stepsize of proximal gradient method

function construct_GD_stepsizes(N)
    α = OffsetArray(zeros(N, N), 1:N, 0:N-1)

    for i in 1:N
        for j in 0:i-1
            α[i,j]=1
        end
    end
     
    return α   
end

## θ sequence

function get_theta(N)
    theta=OffsetArray(zeros(N+1), 0:N)
    theta[0]=1
    for i in 1:N-1
        theta[i]=(1+sqrt(1+4*theta[i-1]^2))/2
    end
    theta[N]=(1+sqrt(1+8*theta[N-1]^2))/2
    return theta
end

## γ sequence

function get_gamma(N)
    γ = OffsetArray(zeros(N), 0:N-1)
    theta = get_theta(N)  
    for i in 0:N-1
        γ[i] = (2 * theta[i] / theta[N]^2) * (theta[N]^2 - 2 * theta[i]^2 + theta[i])
    end

    return γ
end

## construct OGM stepsizes

function ogm_stepsizes(N) 
    theta=get_theta(N)
    α=OffsetArray(zeros(N,N), 1:N, 0:N-1)
    for i in 0:N-1
        for k in 0:i-1
            if k+1<=i
                α[i+1,k]=α[k+1,k]+sum(2*theta[k]/theta[l+1]-(1/theta[l+1])*α[l,k] for l in k+1:i)
            end
        end
        α[i+1,i]=1+(2*theta[i]-1)/theta[i+1]
    end
    return α
end


## construct OptISTA stepsizes

function construct_optista_stepsizes(N)
    α = ogm_stepsizes(N)
    β = ogm_stepsizes(N)
    ϕ = OffsetArray(zeros(N, N), 1:N, 0:N-1)
    ψ = OffsetArray(zeros(N, N), 1:N, 0:N-1)

    for i in 1:N
        for j in 0:i-1
            ϕ[i,j]=α[N,j]
            ψ[i,j]=α[N,j]
        end
    end
     
    return ϕ,ψ,α,β
end

## data generations

function data_generator_function(N, α, β, ϕ, ψ, L)

    dim = (2*N) + 3
    N_pts = (2*N) + 2

    𝐰_star = zeros(dim, 1)
    𝐰_0 = e_i(dim, 1)

    #create gradients
    
    𝐟′ = OffsetArray(zeros(dim, N_pts), 1:dim, -1:2*N)  
    𝐠′ = OffsetArray(zeros(dim, N_pts), 1:dim, -1:2*N)

    # fill the elements of 𝐟′

    for i in -1:N
        𝐟′[:,i] = e_i(dim, i+3)
    end

    # fill the elements of 𝐠′

    𝐠′[:,-1] = -𝐟′[:,-1]

    for i in 1:N
       𝐠′[:,N+i] = e_i(dim, N+3+i)
    end

    #create function evaluations

    𝐟 = OffsetArray(zeros(dim, N_pts), 1:dim, -1:(2*N))
    𝐠 = OffsetArray(zeros(dim, N_pts), 1:dim, -1:(2*N))

    # fill elements of 𝐟

    for i in -1:N
        𝐟[:,i] = e_i(dim, i+2)
    end

    # fill elements of 𝐠

    𝐠[:, -1] = e_i(dim, N+3)

    for i in 1:N
        𝐠[:, N+i] = e_i(dim, N+3+i)
    end

    #make 𝐰
    
    𝐰 = OffsetArray(Matrix{Any}(undef, dim, N_pts), 1:dim, -1:2*N)

    𝐰[:,-1] = 𝐰_star

    𝐰[:, 0] = 𝐰_0

    for i in 1:N
        𝐰[:,N+i] = 𝐰[:, 0] - (1/L)*sum(ϕ[i,j]*𝐟′[:,j] for j in 0:i-1) - (1/L)*sum(ψ[i,j]*𝐠′[:,N+j+1] for j in 0:i-1)
    end
    
    for i in 1:N
        𝐰[:, i] = 𝐰[:,0] - (1/L)*sum(α[i,j]*𝐟′[:,j] for j in 0:i-1) - (1/L)*sum(β[i,j]*𝐠′[:,N+j+1] for j in 0:i-1)
    end

    return 𝐰, 𝐟′, 𝐠′, 𝐟, 𝐠

end

##define matrices
function A_mat_f(i, j, α, β, ϕ, ψ, 𝐟′, 𝐰)  
    return ⊙(𝐟′[:,j], 𝐰[:,i] - 𝐰[:,j])
end

function A_mat_g(i, j, α, β, ϕ, ψ, 𝐠′, 𝐰)  
    return ⊙(𝐠′[:,j], 𝐰[:,i] - 𝐰[:,j])
end

function B_mat(i, j, α, β, ϕ, ψ, 𝐰)     
    return ⊙(𝐰[:,i] - 𝐰[:,j], 𝐰[:,i] - 𝐰[:,j])
end

function C_mat_f(i, j, 𝐟′)
    return ⊙(𝐟′[:,i]-𝐟′[:,j], 𝐟′[:,i]-𝐟′[:,j])
end

function a_vec_f(i, j, 𝐟)
    return 𝐟[:,j]-𝐟[:,i]
end

function a_vec_g(i, j, 𝐠)
    return 𝐠[:,j]-𝐠[:,i]
end


## indices

struct i_j_idx 
    i::Int64 
    j::Int64 
end


function index_set_constructor(N)

    # construct idx_f
    idx_f = [i for i in -1:N]

    # construct idx_g
    idx_g = [-1; [N+i for i in 1:N]]

    # construct the index set for λ
    idx_set_λ = i_j_idx[]
    for i in idx_f
        for j in idx_f
            if i!=j
                push!(idx_set_λ, i_j_idx(i,j))
            end
        end
    end

    # construct the index set for τ
    idx_set_τ = i_j_idx[]
    for i in idx_g
        for j in idx_g
            if i!=j
                push!(idx_set_τ, i_j_idx(i,j))
            end
        end
    end

    return idx_set_λ, idx_set_τ

end


## primal SDP 
function solve_inner_primal(N, L, α, β, ϕ, ψ, R; show_output = :off)

    𝐰, 𝐟′, 𝐠′, 𝐟, 𝐠 = data_generator_function(N, α, β, ϕ, ψ, L)

    dim_G = (2*N)+3
    dim_F = (2*N)+3
    idx_set_λ, idx_set_τ = index_set_constructor(N)
    
    #model
    model_primal = Model(optimizer_with_attributes(Mosek.Optimizer))

    #variables

    @variable(model_primal, F[1:dim_F])
    @variable(model_primal, G[1:dim_G, 1:dim_G], PSD)

    # objective

    @objective(model_primal, Max, F'*(a_vec_f(-1, N, 𝐟) + a_vec_g(-1, 2*N, 𝐠)))

    # constraints

    @constraint(model_primal, tr(G*B_mat(0, -1, α, β, ϕ, ψ, 𝐰)) <= R^2)

    for i_j_λ in idx_set_λ
        @constraint(model_primal,  
        F'*a_vec_f(i_j_λ.i, i_j_λ.j, 𝐟) + tr(G* ( A_mat_f(i_j_λ.i, i_j_λ.j, α, β, ϕ, ψ, 𝐟′, 𝐰) + (1/(2*L)) * C_mat_f(i_j_λ.i, i_j_λ.j, 𝐟′) ))  <= 0 )
    end

    for i_j_τ in idx_set_τ
        @constraint(model_primal,
        F'*a_vec_g(i_j_τ.i, i_j_τ.j, 𝐠) +tr(G*A_mat_g(i_j_τ.i, i_j_τ.j, α, β, ϕ, ψ, 𝐠′, 𝐰)) <= 0)
    end


    #optimize

    set_silent(model_primal)
    optimize!(model_primal)

    # store and return 
    obj_val_primal = objective_value(model_primal)
    G_star = value.(G)
    F_star = value.(F)

    @info "performance measure of primal is $(obj_val_primal)"
    return obj_val_primal, G_star, F_star

end

solve_inner_primal (generic function with 1 method)

In [3]:
# PEP result

N=5
L=2
R=3

ϕ,ψ,α,β = construct_optista_stepsizes(N)

## primal
obj_val_primal, G_star_primal, F_star_primal = solve_inner_primal(N, L, α, β, ϕ, ψ, R; show_output = :off)


┌ Info: performance measure of primal is 0.3475053883064591
└ @ Main In[2]:267


(0.3475053883064591, [8.999999929267846 0.010832212201259791 … 0.6841703191083359 0.6841705145085188; 0.010832212201259791 0.04187172254114919 … -0.04103488926245005 -0.041034892407911874; … ; 0.6841703191083359 -0.04103488926245005 … 0.09386790801059927 0.09386792143060715; 0.6841705145085188 -0.041034892407911874 … 0.09386792143060715 0.09386794386358055], [-0.44586023217820453, 0.7394132335083513, 0.19202287032529236, -0.0344839733283084, -0.14685053886746377, -0.12804422288217318, -0.17619710400231323, -0.1583242196853017, 0.29730260231481004, 0.08314801272725211, -0.04614846926383797, -0.09549599490409498, -0.08048195955473392])

In [4]:
# Theorem 1

θ = get_theta(N)

convergence = L * R^2 / (2*θ[N]^2 - 2)

println("*** worst-case performance of OptISTA ***")
@printf("\t PEP guarantee:\t F(y_N)-F_* <= %.6f ||x0 - xs||^2\n", obj_val_primal)
@printf("\t Theorem 1:\t F(y_N)-F_* <= %.6f ||x0 - xs||^2\n", convergence)


*** worst-case performance of OptISTA ***
	 PEP guarantee:	 F(y_N)-F_* <= 0.347505 ||x0 - xs||^2
	 Theorem 1:	 F(y_N)-F_* <= 0.347505 ||x0 - xs||^2
